In [1]:
import numpy as np
from PIL import Image
import IPython
import os
import sys
import time
import gdown

## INSTALL

In [ ]:
!mkdir ACGPN
%cd ACGPN
!mkdir Data_preprocessing
!mkdir Data_preprocessing/test_color
!mkdir Data_preprocessing/test_edge
!mkdir Data_preprocessing/test_img
!mkdir inputs
!mkdir inputs/img
!mkdir inputs/cloth

%cd ..

#옷 마스킹 모델 파일 다운로드
!git clone https://github.com/levindabhi/U-2-Net.git

#U-2-Net 모델을 돌릴 때 필요한 파일 준비를 위해 해당 모델 폴더로 이동
%cd U-2-Net

#U-2-Net 모델에 필요한 폴더 생성
!mkdir saved_models
!mkdir saved_models/u2net
!mkdir saved_models/u2netp


#사전 학습된 U-2-Net 모델 다운로드
!gdown 1rbSTGKAE-MTxBYHd-51l2hMOQPT_7EPy -O saved_models/u2netp/u2netp.pth
!gdown 1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ -O saved_models/u2net/u2net.pth

%cd ..

# Flow-style-vton 설치
!git clone https://github.com/SenHe/Flow-Style-VTON.git

%cd Flow-Style-VTON

%cd test
!mkdir flow-style-vton-data
!mkdir flow-style-vton-data/test_clothes
!mkdir flow-style-vton-data/test_edge
!mkdir flow-style-vton-data/test_img
%cd ..

!mkdir saved_model
!gdown 1pYrLujkd2gmQGqtnROCzSSnVwbMh9DnP -O saved_model/model.zip
%cd saved_model
!unzip model.zip

%cd ..


%cd ..

# Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop

import shutil

upload_folder = 'upload'
result_folder = 'results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

%cd ..

## U-2Net

In [3]:
%cd U-2-Net

import u2net_load
import u2net_run

u2net = u2net_load.model(model_name='u2netp')

%cd ..

/home/ssrlab/kw/taeyoon/COSMIC-tryon/Model/Flow_style_vton/U-2-Net
...load U2NEP---4.7 MB
/home/ssrlab/kw/taeyoon/COSMIC-tryon/Model/Flow_style_vton


## 중간 작업

In [6]:
#ACGPN 폴더에서 시작해야함
%cd ACGPN

#옷 데이터셋이 올바른 위치에 있는지 확인
sorted(os.listdir('inputs/cloth'))

#전처리 할 옷 데이터 이름을 시간으로 지정
cloth_name = f'cloth_{int(time.time())}.png'

#전처리 할 옷 데이터의 파일 위치 확인
cloth_path = os.path.join('inputs/cloth', sorted(os.listdir('inputs/cloth'))[0])

#이미지 할당
cloth = Image.open(cloth_path)

#이미지 사이즈 조절 및 RGB모드로 변환
cloth = cloth.resize((192, 256), Image.BICUBIC).convert('RGB')

cloth.save(os.path.join('Data_preprocessing/test_color', cloth_name)) # 원본 옷 Resizing 된 데이터 넣음

u2net_run.infer(u2net, 'Data_preprocessing/test_color', 'Data_preprocessing/test_edge') # test_edge에 옷 segmentation 데이터 넣음

sorted(os.listdir('inputs/img'))

img_name = f'img_{int(time.time())}.png'

img_path = os.path.join('inputs/img', sorted(os.listdir('inputs/img'))[0])

img = Image.open(img_path)

img = img.resize((192,256), Image.BICUBIC)

img_path = os.path.join('Data_preprocessing/test_img', img_name)

img.save(img_path)

%cd ..

/home/ssrlab/kw/taeyoon/COSMIC-tryon/Model/Flow_style_vton/ACGPN


Generating mask for: cloth_1694154552.png
Saving output at Data_preprocessing/test_edge/cloth_1694154552.png
/home/ssrlab/kw/taeyoon/COSMIC-tryon/Model/Flow_style_vton


In [ ]:
# 데이터 이동 필요

# ACGPN/Data_preprocessing/test_color -> Flow-Style-VTON/test/flow-style-vton-data/test_clothes (옷)
# ACGPN/Data_preprocessing/test_edge -> Flow-Style-VTON/test/flow-style-vton-data/test_edge(옷 segmentation)
# ACGPN/Data_preprocessing/test_img -> Flow-Style-VTON/test/flow-style-vton-data/test_img(사람)

## Flow-Style-VTON

In [ ]:
# Flow-Style-VTON/test/data/aligned_dataset_test.py 파일 수정 필요

In [7]:
%cd Flow-Style-VTON
%cd test

!python test.py --name demo --resize_or_crop None --batchSize 1 --gpu_ids 1 --warp_checkpoint ../saved_model/ckp/non_aug/PFAFN_warp_epoch_101.pth --gen_checkpoint ../saved_model/ckp/non_aug/PFAFN_gen_epoch_101.pth --dataroot ./flow-style-vton-data

/home/ssrlab/kw/taeyoon/COSMIC-tryon/Model/Flow_style_vton/Flow-Style-VTON
/home/ssrlab/kw/taeyoon/COSMIC-tryon/Model/Flow_style_vton/Flow-Style-VTON/test
------------ Options -------------
batchSize: 1
data_type: 32
dataroot: ./flow-style-vton-data
display_winsize: 512
fineSize: 512
gen_checkpoint: ../saved_model/ckp/non_aug/PFAFN_gen_epoch_101.pth
gpu_ids: [1]
input_nc: 3
isTrain: False
loadSize: 512
max_dataset_size: inf
nThreads: 1
name: demo
no_flip: False
norm: instance
output_nc: 3
phase: test
resize_or_crop: None
serial_batches: False
tf_log: False
use_dropout: False
verbose: False
warp_checkpoint: ../saved_model/ckp/non_aug/PFAFN_warp_epoch_101.pth
-------------- End ----------------
CustomDatasetDataLoader
dataset [AlignedDataset] was created
2032
AFWM(
  (image_features): FeatureEncoder(
    (encoders): ModuleList(
      (0): Sequential(
        (0): DownSample(
          (block): Sequential(
            (0): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running

## Real-ESRGAN

In [ ]:
from pathlib import Path

CURRENT_DIR = Path()
q = CURRENT_DIR / 'Real-ESRGAN' / 'upload'
#d = CURRENT_DIR / 'ACGPN' / 'results' / 'test' / 'try-on' / f"{img_name}"


!pwd